![cover](./images/cover.png)

### <center>Options Value</center>

Option pricing means identifying such **premium** that the **buyer is willing to pay**... and the **seller is willing to receive**.

In its most basic terms, the value of an option is commonly decomposed into two parts:
* The **intrinsic value**, which represents the value of **exercising now** the option
* The **time value**, which represents the value of the **uncertainty** surrounding the **future underlying price**.

![values](./images/values.png)

In [1]:
import numpy as np

import sympy as sy
import plotly
import scipy
import scipy.stats as si
import pandas as pd

import chart_studio.plotly as py
import plotly.offline as po
import plotly.graph_objs as go
from plotly import subplots

import ipywidgets as widgets
from ipywidgets import interact

from ipywidgets.embed import embed_minimal_html

po.init_notebook_mode(connected=True)

In [2]:
def european_option(S, K, T, r, s, option_type = "call"):
    #S: underlying spot price
    #K: strike price
    #T: time to maturity
    #r: risk-free interest rate
    #s: volatility of underlying asset
    
    #omega: option sign (+1 for call, -1 for put)
    option_type = option_type.lower()
    omega = 1 if option_type == "call" else -1

    
    if np.abs(T)<1e-5:
        return np.maximum(omega*(S-K),0.)
   
    d1 = (np.log(S / K) + (r + 0.5 * s ** 2) * T) / (s * np.sqrt(T))
    d2 = (np.log(S / K) + (r - 0.5 * s ** 2) * T) / (s * np.sqrt(T))
       
    premium =(omega * S * scipy.stats.norm.cdf(omega * d1,0.0,1.0)) - (omega * K * np.exp(-r * T) * scipy.stats.norm.cdf(omega * d2,0.0,1.0))
    
    return premium

def option_delta(S, K, T, r, s, option_type = "call"):
    d1 = (np.log(S / K) + (r + 0.5 * s ** 2) * T) / (s * np.sqrt(T))
    if option_type == "call":
        return si.norm.cdf(d1,0.0,1.0)
    elif option_type == "put":
        return (si.norm.cdf(d1,0.0,1.0)-1)
    else:
        print("Error in european_option: invalid option_type "+ str(option_type))
        return 0
    
def option_gamma(S, K, T, r, s, option_type = "call"):
    d1 = (np.log(S / K) + (r + 0.5 * s ** 2) * T) / (s * np.sqrt(T))
    return (si.norm.pdf(d1,0.0,1.0) / (S*s*np.sqrt(T)))

In [3]:
european_option(100.,100.,1,0.03,0.4,'call')

17.138735220515535

In [4]:
layout = widgets.Layout(width='98%',height='50px')
style = {'description_width': 'initial'}

x = np.linspace(30,60,121)

#Creates widget containing a figure
fig1 = go.FigureWidget()
#Adds two different scatter plots
fig1.add_scatter()
fig1.add_scatter()
premium = fig1.data[0]
payoff = fig1.data[1]
#Sets x values
premium.x= payoff.x = x
payoff.y=european_option(x,45.,0.,0.01,0.1,'call')
#Sets names
premium.name = 'Premium'
payoff.name = 'Payoff'

premium.hoverinfo = payoff.hoverinfo = 'y'
premium.hovertemplate = '<b>Premium:</b> %{y:.2f}<extra></extra>'

payoff.hovertemplate = '<b>Payoff:</b> %{y:.2f}<extra></extra>'
#premium.hovertemplate = '<b>Premium:</b> %{y:.2f} <br>Time value: %{text}'
#text = ['Time Value: {:.2f}'.format(premium.y[i]-payoff.y[i]) for i in range(len(payoff.y))]

premium.line=dict(color='Green', width=1.5)
payoff.line=dict(color='Red', width=1.5)

#---------------------------------------------Defines utility widgets-------------------------------------------------#
#Slider and Value box to move exercise date
maturitySlide = widgets.FloatSlider(value = 0, min=0, max=60, step=0.25, description='Time to exercise: ', disabled=False,
    continuous_update=False, orientation='horizontal', readout=False, readout_format='.2f',
    layout=widgets.Layout(width='100%', height='50px'), style=style)
maturityVal = widgets.FloatText(value = 0, step= 0.25,disabled = False)

maturity = widgets.Box([maturitySlide,maturityVal])
maturityLink = widgets.jslink((maturitySlide,'value'), (maturityVal, 'value'))

reset = widgets.Button(description='Reset')

box= widgets.VBox([maturity,reset])

def on_btn_click(change):
    maturitySlide.value=0
    return
reset.on_click(on_btn_click)


#Updating function
def f(maturityVal):
    with fig1.batch_update():
        premium.y=european_option(x,45.,maturityVal,0.01,0.1,'call')
        premium.name='Premium (T='+str(maturityVal)+')'
        #premium.hovertemplate = '<b>Premium:</b> %{y:.2f} <br>Time value: %{text}'
        #text = ['Time Value: {:.2f}'.format(premium.y[i]-payoff.y[i]) for i in range(len(payoff.y))]

out = widgets.interactive_output(f, {'maturityVal': maturityVal})
fig1.add_scatter()
fig1.add_scatter()
iv = fig1.data[2]
tv = fig1.data[3]

iv.line = dict(color='Red',width = 1.5,dash='dot')
iv.opacity = tv.opacity = 0.5
iv.mode = tv.mode = 'lines'
tv.line = dict(color='Green',width = 1.5,dash='dot')
iv.name = 'Intrinsic Value'
tv.name = 'Time Value'
tv.hoverinfo = iv.hoverinfo = 'none'

tv.showlegend = iv.showlegend = False
# create our callback function
def update_point(trace, points, selector):
    i = points.point_inds
    with fig1.batch_update():
        iv.x = [points.xs[0],points.xs[0]]
        iv.y = [0,payoff.y[i][0]]
        tv.x = [points.xs[0],points.xs[0]]
        tv.y = [payoff.y[i][0],premium.y[i][0]]
        fig1['layout']['annotations'] = [dict(x=points.xs[0]+2,
                                           y=points.ys[0]/2,
                                           visible = (points.ys[0] != 0),
                                           xref='x',yref='y',
                                           text='Intrinsic Value',
                                           showarrow=False,
                                           font={'color':'Red'},
                                           opacity=0.8,
                                           bgcolor='White'),
                                      dict(x=points.xs[0]+2,
                                           y=(premium.y[i][0] + payoff.y[i][0])/2,
                                           visible = (premium.y[i][0].item() != payoff.y[i][0].item()),
                                           xref='x',yref='y',
                                           text='Time Value',
                                           showarrow=False,
                                           font={'color':'Green'},
                                           opacity=0.8,
                                           bgcolor='White')]
        

premium.on_hover(update_point)
#premium.on_click(update_point)
#Sets figure title and axis 
fig1.layout.title={'text':'Options Premium for different times to exercise','font':{'size': 25},'x':0.5,'xanchor':'center'}
fig1.layout.yaxis={'range': [0,20],'title': 'Option premium (€)','titlefont':{'size': 15}, 'hoverformat':',.2f'}
fig1.layout.xaxis={'range':[30,60],'title': 'Underlying spot price (€)', 'tick0':0, 'dtick':5, 'titlefont':{'size': 15}, 'tickformat':',.2f',}
fig1['layout']['showlegend'] = True
fig1['layout']['height'] = 700
fig1['layout']['width'] = 1100
fig1['layout']['hovermode'] = 'x'

In [5]:
fig1.update_layout(legend={'orientation':'h'})
display(box, out, fig1)

Output()

FigureWidget({
    'data': [{'hoverinfo': 'y',
              'hovertemplate': '<b>Premium:</b> %{y:.2f}<extra>…

### <center>From binomial tree to Black-Scholes-Merton model</center>

In the limit, i.e. for *N* that goes to infinity, and if *u* and *d* are consistent with generating a log-normal distribution for the underlying price, the **binomial tree model converges to the Black-Scholes-Merton option pricing model** that gives:

$$ \pi_{call} (t) = S_0 \cdot \textbf{N} (d_1) - K \cdot e^{-r\cdot t} \cdot \textbf{N}(d_2)$$

$$ \pi_{put} (t) = K \cdot e^{-r\cdot t} \cdot \textbf{N}(- d_2) - S_0 \cdot \textbf{N} (- d_1)$$

where:

* $d_1 = \frac{1}{\sigma \sqrt{t}}\cdot\left[ \ln{\left(\frac{S_0}{K}\right)} + \left( r + \frac{\sigma^2}{2}\right)\cdot t \right]$
* $d_2 = d_1 - \sigma \sqrt{t}$
* $\textbf{N}\left(\cdot \right)$ is the standard normal cumulative distribution function.

In [6]:
model_comparison = pd.read_csv('tree2BlackScholes.csv', delimiter=';')

In [7]:
tree_scatter = go.Scatter(y=model_comparison['tree_price'],name='Binomial tree', mode='lines')
bs_scatter = go.Scatter(y=model_comparison['bs_price'],name='Black-Scholes-Merton', mode='lines')
elapsed_scatter = go.Scatter(y=model_comparison['elapsed'],name='Elapsed', mode='lines')

prices = subplots.make_subplots(specs=[[{'secondary_y':True}]])
prices.add_trace(tree_scatter,secondary_y=False)
prices.add_trace(bs_scatter,secondary_y=False)
prices.add_trace(elapsed_scatter,secondary_y=True)

# Update xaxis properties
prices.update_xaxes(title='Iteration',titlefont={'size': 10})

# Update yaxis properties
prices.update_yaxes(title='Call premium (€)',tickformat='.2f', hoverformat='.4f',secondary_y=False)
prices.update_yaxes(title='Elapsed (seconds)',tickformat='.2f',secondary_y=True)


prices['layout']['title'] = {'text':'Models convergence','font':{'size': 25},'x':0.5,'xanchor':'center'}#
prices['layout']['height'] = 1000
prices['layout']['width'] = 1500
prices['layout']['hovermode'] = 'x'
prices.update_layout(legend={'orientation':'h'})
prices['layout']['showlegend'] = False

In [8]:
po.iplot(prices)

### <center>Factors affecting option price</center>

There are several factors that affect the option value. Changing one of them at time, the options value shows a specific behavior.

In [9]:
layout = widgets.Layout(width='95%', height='50px')
style = {'description_width': '18%'}

x = np.linspace(10,110,1001)

#Creates widget containing a figure
fig2 = go.FigureWidget()
#Adds two different scatter plots
fig2.add_scatter()
fig2.add_scatter()
premium2= fig2.data[0]
payoff2 = fig2.data[1]
#Sets x values
premium2.x=x
payoff2.x=x
#Sets names
premium2.name = 'Premium'
payoff2.name = 'Payoff'

#---------------------------------------------Defines widgets-------------------------------------------------#
#Slider and Value box to move spot prive
spotSlide = widgets.FloatSlider(value = 45, min=1, max=100, step=0.1, description='Spot value:', disabled=False,
    continuous_update=False, orientation='horizontal', readout=False, readout_format='.2f', layout=layout, style=style)
spotVal = widgets.FloatText(value = 45, step= 0.1,disabled = False)

spot = widgets.Box([spotSlide,spotVal])
spotLink = widgets.jslink((spotSlide,'value'), (spotVal, 'value'))

#Slider and Value box to move strike
strikeSlide = widgets.FloatSlider(value = 45, min=1, max=100, step=0.1, description='Option Strike: ', disabled=False,
    continuous_update=False, orientation='horizontal', readout=False, readout_format='.2f', layout=layout, style=style)
strikeVal = widgets.FloatText(value = 45, step= 0.1,disabled = False)

strike = widgets.Box([strikeSlide,strikeVal])
strikeLink = widgets.jslink((strikeSlide,'value'), (strikeVal, 'value'))

#Slider and Value box to move exercise date
matySlide = widgets.FloatSlider(value = 0, min=0, max=60, step=0.25, description='Time to exercise: ', disabled=False,
    continuous_update=False, orientation='horizontal', readout=False, readout_format='.2f', layout=layout, style=style)
matyVal = widgets.FloatText(value = 0, step= 0.25,disabled = False)

maty = widgets.Box([matySlide,matyVal])
matyLink = widgets.jslink((matySlide,'value'), (matyVal, 'value'))

#Slider and Value box to move risk-free rate
rateSlide = widgets.FloatSlider(value = 0.02, min=-0.05, max=0.1, step=0.001, description='Risk-free rate: ', disabled=False,
    continuous_update=False, orientation='horizontal', readout=False, readout_format='.2p', layout=layout, style=style)
rateVal = widgets.FloatText(value = 0.02, step= 0.001,disabled = False)

rate = widgets.Box([rateSlide, rateVal])
rateLink = widgets.jslink((rateSlide,'value'),(rateVal, 'value'))

#Slider anda Value box to move volatility
sigmaSlide = widgets.FloatSlider(value = 0.1, min=0.001,max=1, step=0.001, description='Underlying volatility: ', disabled=False,
    continuous_update=False, orientation='horizontal', readout=False, readout_format='.2p', layout=layout, style=style)
sigmaVal = widgets.FloatText(value = 0.1, step= 0.001,disabled = False,readout_format='.2p')

sigma = widgets.Box([sigmaSlide,sigmaVal])
sigmaLink = widgets.jslink((sigmaSlide,'value'),(sigmaVal, 'value'))

#Dropdown menu to choose option typology (Call / Put)
dropdwn = widgets.Dropdown(
    options=['call', 'put'],
    value='call',
    #description='Option Type:',
    disabled=False)
#Button to restore default values
reset = widgets.Button(description='Reset')

hbox= widgets.HBox([dropdwn,reset])
box= widgets.VBox([spot,strike,maty,rate, sigma,hbox])

def on_btn_click(change):
    spotSlide.value=45
    strikeSlide.value=45
    matySlide.value=0
    rateSlide.value=0.02
    sigmaSlide.value=0.1
    dropdwn.value='call'
    return
reset.on_click(on_btn_click)

#Updating function
def f(strikeVal, matyVal,rateVal,sigmaVal,dropdwn):
    with fig2.batch_update():
        premium2.x=x
        premium2.y=european_option(x,strikeVal, matyVal,rateVal,sigmaVal,dropdwn)
        payoff2.y=european_option(x,45.,0.,0.01,0.1,dropdwn)

out = widgets.interactive_output(f, {'strikeVal': strikeVal, 'matyVal': matyVal, 'rateVal': rateVal,
                                     'sigmaVal': sigmaVal, 'dropdwn': dropdwn})


#Sets figure title and axis 
fig2.layout.title={'text':'Options Premium changes','font':{'size': 25},'x':0.5,'xanchor':'center'}
fig2.layout.yaxis={'range': [0,30],'title': 'Option premium (€)','titlefont':{'size': 10},'hoverformat':',.4f'}
fig2.layout.xaxis={'range':[10,80],'title': 'Underlying spot price (€)', 'tick0':0, 'dtick':5, 'titlefont':{'size': 10}, 'tickformat':',.0f','hoverformat':',.2f'}
fig2['layout']['showlegend'] = True
fig2['layout']['height'] = 700
fig2['layout']['width'] = 1500
fig2['layout']['hovermode'] = 'x'

In [10]:
#Displays widgets
fig2.update_layout(legend={'orientation':'h'})
display(box, out,fig2)

Output()

FigureWidget({
    'data': [{'name': 'Premium',
              'type': 'scatter',
              'uid': 'c049a1c…

### <center>Hedging strategy: Delta neutral portfolio</center>

Taking position in options exposes the parties to gain/loss due to the (random) variation of the market.
To hedge an option (e.g. a call option) with a Delta exposure of $\Delta$ written on $x$ units of underlying, the investor should take a position of $-x \cdot \Delta$ in the underlying asset.

In [15]:
spot = np.linspace(1,101,1001)
option1_premium = [european_option(x,45.,10,0.001,0.05,'call') for x in spot]

delta=option_delta(45.,45.,10,0.001,0.05,'call')

delta_fig=go.FigureWidget()

# 1 - Option scatter
delta_fig.add_scatter()
opt_scatter = delta_fig.data[0]
opt_scatter.x=spot
opt_scatter.y=option1_premium
opt_scatter.name='Option#1'
opt_scatter.line=dict(dash='dash',width=1.5)
opt_scatter.opacity=.5
opt_scatter.visible=True

# 2 - Asset scatter
asset_price=[(45. -x)*delta for x in spot]
delta_fig.add_scatter()
asset_scatter = delta_fig.data[1]
asset_scatter.x = spot
asset_scatter.y = asset_price
asset_scatter.name='Underlying#1'
asset_scatter.line=dict(dash='dash', width=1.5)
asset_scatter.opacity=.5
asset_scatter.visible=False


# 3 - Portfolio 1 scatter
ptf1=option1_premium
delta_fig.add_scatter()
ptf1_scatter=delta_fig.data[2]
ptf1_scatter.x=spot
ptf1_scatter.y=ptf1
ptf1_scatter.name='Portfolio'
ptf1_scatter.line=dict(color='Green',width=1.5)
ptf1_scatter.opacity=1
ptf1_scatter.visible=False

# 4 - Portfolio 2 scatter
ptf2=[a+b for a,b in zip(option1_premium, asset_price)]
delta_fig.add_scatter()
ptf2_scatter=delta_fig.data[3]
ptf2_scatter.x=spot
ptf2_scatter.y=ptf2
ptf2_scatter.name='Portfolio'
ptf2_scatter.line=dict(color='Green',width=1.5)
ptf2_scatter.opacity=1
ptf2_scatter.visible=False

# Add dropdown
delta_fig.layout.updatemenus=[
        dict(
            type = "buttons",
            direction = "left",
            buttons=list([
                dict(
                    args=[{'visible':[True,False,True,False]},
                         {'annotations':[{'xref':'paper', 'yref':'paper','x':0.01, 'y':0.95, 'text':r'$\text{Step 1: Take position in a call option with a Delta exposure of }\Delta \text{ written on }x \text{ units of underlying}$','showarrow':False}]}],
                    label="Step 1",
                    method="update"
                ),
                dict(
                    args=[{'visible':[True,True,False,True]},
                         {'annotations':[{'xref':'paper', 'yref':'paper','x':0.01, 'y':0.95, 'text':r'$\text{Step 1: Take position in a call option with a Delta exposure of }\Delta \text{ written on }x \text{ units of underlying}$','showarrow':False},
                                        {'xref':'paper', 'yref':'paper','x':0.01, 'y':0.90, 'text':r'$\text{Step 2: Take a position of }-x \cdot \Delta \text{ in the underlying asset}$','showarrow':False}]}],
                    label="Step 2",
                    method="update"
                )
            ]),
            pad = {'r': 10, 't': 87},
            showactive = True,
            x = 1.0,
            xanchor = 'right',
            y = 0.05,
            yanchor = 'top')]
delta_fig.layout.showlegend = True
delta_fig['layout']['height'] = 700
delta_fig['layout']['width'] = 1100
delta_fig.layout.title={'text':'Delta hedging strategy','font':{'size': 25},'x':0.5,'xanchor':'center'}
delta_fig.layout.yaxis={'range': [-30,30],'fixedrange': False, 'title': 'Strategy value (€)','titlefont':{'size': 10}, 'hoverformat':',.4f'}
delta_fig.layout.xaxis={'range':[10,80],'title': 'Underlying spot price (€)', 'tick0':0, 'dtick':5, 'titlefont':{'size': 10}, 'tickformat':',.1f',}

In [18]:
delta_fig.update_layout(legend={'orientation':'h'})
delta_fig

FigureWidget({
    'data': [{'line': {'dash': 'dash', 'width': 1.5},
              'name': 'Option#1',
       …

### <center>Hedging strategy: Gamma neutral portfolio</center>

Suppose that an investor wants to hedge (Delta/Gamma) his position in a option written on 𝑥 units of underlying. He should:

* Take position in an option $O_1$
* Compute the option Delta $\Delta_1$ and take a position of $-x \cdot \Delta_1$ in the underlying asset
* Compute the portfolio ($O_1$ + underlying) Gamma $\Gamma_2$ and take a position of $\omega$ in another option $O_2$ with Gamma equals to $\Gamma_3$ such that $\omega = \frac{\Gamma_2}{\Gamma_3}$
* Compute the portfolio ($O_1$ + underlying + $O_2$) Delta $\Delta_3$ and rebalance his position in the underlying asset to hold $-x \cdot \Delta_3$ units.

In [19]:
spot = np.linspace(1,101,1001)
option1_premium = [european_option(x,45.,10,0.001,0.05,'call') for x in spot]

delta=option_delta(45.,45.,10,0.001,0.05,'call')
gamma=option_gamma(45.,45.,10,0.001,0.05,'call')

gamma2=-option_gamma(45.,40.,10,0.001,0.05,'put')

ratio=gamma/gamma2

option2_premium = [ratio*european_option(x,40,10,0.001,0.05,'put') for x in spot]


delta2=ratio*option_delta(45.,40.,10,0.001,0.05,'put')

gamma_fig=go.FigureWidget()

# 1 - Option scatter
gamma_fig.add_scatter()
opt_scatter = gamma_fig.data[0]
opt_scatter.x=spot
opt_scatter.y=option1_premium
opt_scatter.name='Option#1'
opt_scatter.line=dict(dash='dash',width=1.5)
opt_scatter.opacity=.5
opt_scatter.visible=True

# 2 - Asset scatter
asset_price=[(45. -x)*delta for x in spot]
gamma_fig.add_scatter()
asset_scatter = gamma_fig.data[1]
asset_scatter.x = spot
asset_scatter.y = asset_price
asset_scatter.name='Underlying#1'
asset_scatter.line=dict(dash='dash', width=1.5)
asset_scatter.opacity=.5
asset_scatter.visible=False

# 3 - New option scatter
new_opt=option2_premium
gamma_fig.add_scatter()
new_opt_scatter=gamma_fig.data[2]
new_opt_scatter.x=spot
new_opt_scatter.y=new_opt
new_opt_scatter.name='Option#2'
new_opt_scatter.line=dict(dash='dash',width=1.5)
new_opt_scatter.opacity=.5
new_opt_scatter.visible=False

# 4 - New asset scatter
new_asset_price=[(45.-x)*delta2 for x in spot]
gamma_fig.add_scatter()
new_asset_scatter = gamma_fig.data[3]
new_asset_scatter.x = spot
new_asset_scatter.y = new_asset_price
new_asset_scatter.name='Underlying#2'
new_asset_scatter.line=dict(dash='dash', width=1.5)
new_asset_scatter.opacity=.5
new_asset_scatter.visible=False

# 5 - Portfolio 1 scatter
ptf1=option1_premium
gamma_fig.add_scatter()
ptf1_scatter=gamma_fig.data[4]
ptf1_scatter.x=spot
ptf1_scatter.y=ptf1
ptf1_scatter.name='Portfolio'
ptf1_scatter.line=dict(color='Green',width=1.5)
ptf1_scatter.opacity=1
ptf1_scatter.visible=False

# 6 - Portfolio 2 scatter
ptf2=[a+b for a,b in zip(option1_premium, asset_price)]
gamma_fig.add_scatter()
ptf2_scatter=gamma_fig.data[5]
ptf2_scatter.x=spot
ptf2_scatter.y=ptf2
ptf2_scatter.name='Portfolio'
ptf2_scatter.line=dict(color='Green',width=1.5)
ptf2_scatter.opacity=1
ptf2_scatter.visible=False

# 7 - Portfolio 3 scatter
ptf3=[a+b for a,b in zip(ptf2,new_opt)]
gamma_fig.add_scatter()
ptf3_scatter=gamma_fig.data[6]
ptf3_scatter.x=spot
ptf3_scatter.y=ptf3
ptf3_scatter.name='Portfolio'
ptf3_scatter.line=dict(color='Green',width=1.5)
ptf3_scatter.opacity=1
ptf3_scatter.visible=False

# 8- Portfolio 4 scatter
ptf4=[a+b for a,b in zip(ptf3,new_asset_price)]
gamma_fig.add_scatter()
ptf4_scatter=gamma_fig.data[7]
ptf4_scatter.x=spot
ptf4_scatter.y=ptf4
ptf4_scatter.name='Portfolio'
ptf4_scatter.line=dict(color='Green',width=1.5)
ptf4_scatter.opacity=1
ptf4_scatter.visible=False

# Add dropdown
gamma_fig.layout.updatemenus=[
        dict(
            type = "buttons",
            direction = "left",
            buttons=list([
                dict(
                    args=[{'visible':[True,False,False,False,True,False,False,False]},
                         {'annotations':[{'xref':'paper', 'yref':'paper','x':0.01, 'y':0.95, 'text':r'$\text{Step 1: Take position in an option }O_1$','showarrow':False}]}],
                    label="Step 1",
                    method="update"
                ),
                dict(
                    args=[{'visible':[True,True,False,False,False,True,False,False]},
                         {'annotations':[{'xref':'paper', 'yref':'paper','x':0.01, 'y':0.95, 'text':r'$\text{Step 1: Take position in an option }O_1$','showarrow':False},
                {'xref':'paper', 'yref':'paper','x':0.01, 'y':0.90, 'text':r'$\text{Step 2: Compute the option Delta}\Delta_1 \text{ and take positioin of }-x \cdot \Delta_1 \text{ in the underlying asset}$','showarrow':False}]}],
                    label="Step 2",
                    method="update"
                ),
                dict(
                    args=[{'visible':[True,True,True,False,False,False,True,False]},
                         {'annotations':[{'xref':'paper', 'yref':'paper','x':0.01, 'y':0.95, 'text':r'$\text{Step 1: Take position in an option }O_1$','showarrow':False},
                {'xref':'paper', 'yref':'paper','x':0.01, 'y':0.90, 'text':r'$\text{Step 2: Compute the option Delta}\Delta_1 \text{ and take positioin of }-x \cdot \Delta_1 \text{ in the underlying asset}$','showarrow':False},
                {'xref':'paper', 'yref':'paper','x':0.01, 'y':0.85, 'text':r'$\text{Step 3: Compute the portfolio }(O_1 + underlying) \text{ Gamma } \Gamma_2 \text{ and take a position of }\omega \text{ in another option }O_2 \text{ with Gamma equals to }\Gamma_3 \text{ such that }\omega = \frac{\Gamma_2}{\Gamma_3}$','showarrow':False}]}],
                    label="Step 3",
                    method="update"
                ),
                dict(
                    args=[{'visible':[True,True,True,True,False,False,False,True]},
                         {'annotations':[{'xref':'paper', 'yref':'paper','x':0.01, 'y':0.95, 'text':r'$\text{Step 1: Take position in an option }O_1$','showarrow':False},
                {'xref':'paper', 'yref':'paper','x':0.01, 'y':0.90, 'text':r'$\text{Step 2: Compute the option Delta}\Delta_1 \text{ and take positioin of }-x \cdot \Delta_1 \text{ in the underlying asset}$','showarrow':False},
                {'xref':'paper', 'yref':'paper','x':0.01, 'y':0.85, 'text':r'$\text{Step 3: Compute the portfolio }(O_1 + underlying) \text{ Gamma } \Gamma_2 \text{ and take a position of }\omega \text{ in another option }O_2 \text{ with Gamma equals to }\Gamma_3 \text{ such that }\omega = \frac{\Gamma_2}{\Gamma_3}$','showarrow':False},
                {'xref':'paper', 'yref':'paper','x':0.01, 'y':0.80, 'text':r'$\text{Step 4: Compute the portfolio }(O_1 + underlying + O_2) \text{ Delta }\Delta_3 \text{ and rebalance his position in the underlying asset to hold }-x \cdot \Delta_3\text{  units}$','showarrow':False}]}],
                    label="Step 4",
                    method="update"
                )
            ]),
            pad = {'r': 10, 't': 87},
            showactive = True,
            x = 1.0,
            xanchor = 'right',
            y = 0.05,
            yanchor = 'top')]

gamma_fig.layout.showlegend = True
gamma_fig['layout']['height'] = 700
gamma_fig['layout']['width'] = 1100
gamma_fig.layout.title={'text':'Gamma hedging strategy','font':{'size': 25},'x':0.5,'xanchor':'center'}
gamma_fig.layout.yaxis={'range': [-30,30],'fixedrange': False, 'title': 'Strategy value (€)','titlefont':{'size': 10}, 'hoverformat':',.4f'}
gamma_fig.layout.xaxis={'range':[10,80],'title': 'Underlying spot price (€)', 'tick0':0, 'dtick':5, 'titlefont':{'size': 10}, 'tickformat':',.1f',}

In [20]:
gamma_fig.update_layout(legend={'orientation':'h'})
gamma_fig 

FigureWidget({
    'data': [{'line': {'dash': 'dash', 'width': 1.5},
              'name': 'Option#1',
       …